In this notebook, we demonstrate how to use the agavepy package to submit and monitor a job to run the opensees application on Stampede.  

First, we need to import the Agave class from the agavepy package.

In [3]:
from agavepy.agave import Agave

With the Agave class imported, we can now instantiate our client. Typically, this would involve passing your OAuth credentials, but because we are in a notebook on JupyterHub, we can use the "restore" shortcut to create a client with credentials already saved for us behind the scenes.

In [21]:
ag = Agave.restore()

With our client instantiated, we have the full Agave API at our disposal. For example, we can look up the OpenSees application and get specific information about its configuration if we know the application id. 

In [47]:
app_id = 'opensees-docker-2.5.0.6248u2'

In [54]:
app = ag.apps.get(appId=app_id)

The app object has lots of useful attributes describing the OpenSees application. Two, of particular interest, are the inputs (files and/or directories required by the application) and parameters (flags, switches and other options controlling application execution) that are involved when submitting a job.

Let's look at both, starting with the inputs.

In [55]:
app.inputs

[{u'details': {u'argument': None,
   u'description': u"The directory containing your OpenSees input files as well as your OpenSees TCL script. You can drag the link for the directory from the Data Browser on the left, or click the 'Select Input' button and then select the directory. To try out sample data copy and paste 'agave://designsafe.storage.default/mock/examples/opensees/FreefieldAnalysisEffective' above.",
   u'label': u'Input Directory',
   u'repeatArgument': False,
   u'showArgument': False},
  u'id': u'inputDirectory',
  u'semantics': {u'fileTypes': [u'raw-0'],
   u'maxCardinality': 1,
   u'minCardinality': 1,
   u'ontology': [u'xsd:string']},
  u'value': {u'default': u'agave://designsafe.storage.default/mock/FreefieldAnalysis-Effective',
   u'enquote': False,
   u'order': 0,
   u'required': True,
   u'validator': u'',
   u'visible': True}}]

The inputs object is a list, in this case of exactly one input.

In [50]:
app.parameters

[{u'details': {u'argument': None,
   u'description': u"The filename only of the OpenSees TCL script to execute. This file should reside in the Input Directory specified. To try this out copy and paste in 'freeFieldEffective.tcl'.",
   u'label': u'TCL Script',
   u'repeatArgument': False,
   u'showArgument': False},
  u'id': u'inputScript',
  u'semantics': {u'maxCardinality': 1,
   u'minCardinality': 1,
   u'ontology': [u'http://sswapmeet.sswap.info/mime/text/Tcl']},
  u'value': {u'default': u'freeFieldEffective.tcl',
   u'enquote': False,
   u'order': 0,
   u'required': True,
   u'type': u'string',
   u'validator': u'([^\\s]+(\\.(?i)(tcl))$)',
   u'visible': True}}]

Similarly, the parameters object is a list, and it contains one parameter.

We see that for this version of OpenSees, one input and one parameter are available, and both are required. The input, inputDirectory, is the directory containing our OpenSees input files, and the parameter, inputScript, is filename only of the OpenSees TCL script to execute. We know are ready to submit a job.

To illustrate launching a job to run OpenSees, we are going to use an example directory that lives on the DesignSafe default storage system and a specific tcl file in that directory. You will replace these with your own directory and file.

In order to submit a job to Agave, you create a dictionary containing a description of the job. There are many required and optional attributes, but the description below provides a minimal example. Here we are defining:
- jobName: a name for our job, can be anything.
- appId: the id of the application we want to run; in this case, the id of the OpenSees app.
- executionSystem: the system where we are running opensees.
- batchQueue: the queue to run in.
- nodeCount: how many nodes on the execution system we want to use for our job.
- memoryPerNode: Amount of memory needed per node (in GB) for this application to run.
- archive: Whether we want Agave to archive the outputs back to our storage system when we are done. In this case, we do want that.
- retries: How many times we want Agave to retry the job submission in case of failures.
- inputs: Here we provide a dictionary describing the inputs we want to use. The dictionary should have one key for each input and the key should be the input's id attribute. The value should be a list of files or directories, as required by the input (some inputs take multiple values). In our case, we have only one input with id "inputDirectory" and we will use a list with a single value referencing our directory.
- parameters: like inputs, the parameters should be a dictionary with a key for each parameter (the parameter id). The value's type depends on the parameter -- it could be string, int, list, etc. In our case, we have just one parameter with id "inputScript", and the value should be a string containing the name of the file.

Here is the full job description dictionary:

In [51]:
job_description = {
    "jobName": "opensees-ex",
    "appId" : "opensees-docker-2.5.0.6248",
    "executionSystem" : "designsafe.community.data-01",
    "batchQueue" : "debug",
    "nodeCount" : 1,
    "processorsPerNode" : 1,
    "memoryPerNode" : 1.0,
    "maxRunTime" : "00:30:00",
    "archive" : True,
    "retries" : 0,
    "inputs" : {
      "inputDirectory" : [ "agave://designsafe.storage.default/mock/examples/opensees/FreefieldAnalysisEffective" ]
    },
    "parameters" : {
      "inputScript" : "freeFieldEffective.tcl"
    }
}

We are now ready to submit our job:

In [56]:
job = ag.jobs.submit(body=job_description)

Note that Agave returns a response (almost) immediately, and we've stored the response in an object called job. This doesn't mean our job is finished (or even that it has started). It just means Agave has received our request and has queued up work to launch our job. The job object has information we can use to get details about the status of our job.

We can also use agavepy's async module to inspect the job's status:

In [58]:
from agavepy.async import AgaveAsyncResponse

First, import the module and then create an AgaveAsyncResponse object by passing in our AGave client and the job object. Note that we could also use this approach to manage other kinds of asynchronous responses from Agave, such as file transfer responses.

In [61]:
asrp = AgaveAsyncResponse(ag, job)

Using the asrp object, we can check whether the job is done:

In [62]:
asrp.done()

True

The done() method returns either True or False immediately. If we want to just programmatically wait for the job to finish, we can do that using the result() method. The result() method takes an optional timeout paramter.

In [63]:
asrp.result()

'FINISHED'

Once the job is done, the output should be archived and waiting for us. We can check all the details of the job be retrieving it using the jobs service:

In [64]:
ag.jobs.get(jobId=job.id)

{u'_links': {u'app': {u'href': u'https://agave.designsafe-ci.org/apps/v2/opensees-docker-2.5.0.6248'},
  u'archiveData': {u'href': u'https://agave.designsafe-ci.org/files/v2/listings/system/designsafe.storage.default/envision/archive/jobs/job-2740144125295268326-242ac114-0001-007'},
  u'archiveSystem': {u'href': u'https://agave.designsafe-ci.org/systems/v2/designsafe.storage.default'},
  u'executionSystem': {u'href': u'https://agave.designsafe-ci.org/systems/v2/designsafe.community.data-01'},
  u'history': {u'href': u'https://agave.designsafe-ci.org/jobs/v2/2740144125295268326-242ac114-0001-007/history'},
  u'metadata': {u'href': u'https://agave.designsafe-ci.org/meta/v2/data/?q=%7B%22associationIds%22%3A%222740144125295268326-242ac114-0001-007%22%7D'},
  u'notifications': {u'href': u'https://agave.designsafe-ci.org/notifications/v2/?associatedUuid=2740144125295268326-242ac114-0001-007'},
  u'owner': {u'href': u'https://agave.designsafe-ci.org/profiles/v2/envision'},
  u'permissions': 

The archivePath shows us where Agave saved our results. Let's look at what's there:

In [68]:
ag.files.listOnDefaultSystem(filePath='envision/archive/jobs/job-2740144125295268326-242ac114-0001-007')

[{u'_links': {u'history': {u'href': u'https://agave.designsafe-ci.org/files/v2/history/system/designsafe.storage.default///envision/archive/jobs/job-2740144125295268326-242ac114-0001-007'},
   u'metadata': {u'href': u'https://agave.designsafe-ci.org/meta/v2/data?q=%7B%22associationIds%22%3A%222042590382148817382-242ac113-0001-002%22%7D'},
   u'self': {u'href': u'https://agave.designsafe-ci.org/files/v2/media/system/designsafe.storage.default///envision/archive/jobs/job-2740144125295268326-242ac114-0001-007'},
   u'system': {u'href': u'https://agave.designsafe-ci.org/systems/v2/designsafe.storage.default'}},
  u'format': u'folder',
  u'lastModified': datetime.datetime(2016, 8, 18, 18, 13, 25, tzinfo=tzoffset(None, -18000)),
  u'length': 32768,
  u'mimeType': u'text/directory',
  u'name': u'.',
  u'path': u'//envision/archive/jobs/job-2740144125295268326-242ac114-0001-007',
  u'permissions': u'ALL',
  u'system': u'designsafe.storage.default',
  u'type': u'dir'},
 {u'_links': {u'self': {u

Cool! It looks like all our output files are there. Now we're ready to explore the outputs and move on to the next phase of our analysis.